# Examples for SRL - allennlp

The examples in this notebook work with the allennlp model. You can easily adapt the code to the output of a different model. 

The main purpose of the code presented here is to adapt Checklist to a complex sequence classification problem. Inspiration is taken from the NER notebook included in checklist. 

I'm sure there are more sophisticated tests - this is just supposed to get you started. 

Enjoy!

In [1]:
from allennlp_models.pretrained import load_predictor

In [3]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [4]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/felixdenheijer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
import allennlp_models
print(allennlp_models.__file__)

/Users/felixdenheijer/opt/anaconda3/envs/NLPT/lib/python3.8/site-packages/allennlp_models/__init__.py


In [5]:
from checklist.pred_wrapper import PredictorWrapper

In [8]:
import logging

In [9]:
# Turn off some of the INFO verbose messages
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').setLevel(logging.INFO) 
logging.getLogger('urllib3.connectionpool').disabled = True 

In [10]:
# load model and inspect output
# allennlp srl model: https://docs.allennlp.org/models/main/models/structured_prediction/models/srl/
# propbank labels
# paper with a bit of error analysis: https://aclanthology.org/P17-1044.pdf

srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("The killer killed the victim with a knife.")
output

2022-03-15 15:44:21,700 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-03-15 15:44:23,433 - WARNING - allennlp.common.model_card - lerc is not a registered model.
2022-03-15 15:44:24,311 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-03-15 15:44:24,568 - INFO - cached_path - cache of https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz is up-to-date
2022-03-15 15:44:24,570 - INFO - allennlp.models.archival - loading archive file https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz from cache at /Users/felixdenheijer/.allennlp/cache/60314a853eb0aaa774d176d878c62469d49872feb4f2bfd071a75c77f6d76707.1b91cc27e347f2df04ce771a304bee2b70a2c487626b67e277d44c593b868c25
2022-03-15 15:44:24,572 - INFO - allennlp.models.archival - extracting archive file /Users/felixdenheijer/.allennlp/cache/60314a853eb0aaa774d176d878c62469d49872feb4f2bfd071a75c77f6d76707.1b91cc27e347f2df04ce771a304be

{'verbs': [{'verb': 'killed',
   'description': '[ARG0: The killer] [V: killed] [ARG1: the victim] [ARG2: with a knife] .',
   'tags': ['B-ARG0',
    'I-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'B-ARG2',
    'I-ARG2',
    'I-ARG2',
    'O']}],
 'words': ['The',
  'killer',
  'killed',
  'the',
  'victim',
  'with',
  'a',
  'knife',
  '.']}

In [12]:
# pylint: disable=invalid-name,protected-access
import logging
import os
import pathlib
import shutil
import tempfile
from unittest import TestCase

from allennlp.common.checks import log_pytorch_version_info

TEST_DIR = tempfile.mkdtemp(prefix="allennlp_tests")

class AllenNlpTestCase(TestCase):  # pylint: disable=too-many-public-methods
    """
    A custom subclass of :class:`~unittest.TestCase` that disables some of the
    more verbose AllenNLP logging and that creates and destroys a temp directory
    as a test fixture.
    """
    PROJECT_ROOT = (pathlib.Path('__file__').parent / ".." / ".." / "..").resolve()  # pylint: disable=no-member
    MODULE_ROOT = PROJECT_ROOT / "allennlp"
    TOOLS_ROOT = MODULE_ROOT / "tools"
    TESTS_ROOT = MODULE_ROOT / "tests"
    FIXTURES_ROOT = TESTS_ROOT / "fixtures"

    def setUp(self):
        logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
                            level=logging.DEBUG)
        # Disabling some of the more verbose logging statements that typically aren't very helpful
        # in tests.
        logging.getLogger('allennlp.common.params').disabled = True
        logging.getLogger('allennlp.nn.initializers').disabled = True
        logging.getLogger('allennlp.modules.token_embedders.embedding').setLevel(logging.INFO)
        logging.getLogger('urllib3.connectionpool').disabled = True
        log_pytorch_version_info()

        self.TEST_DIR = pathlib.Path(TEST_DIR)

        os.makedirs(self.TEST_DIR, exist_ok=True)

    def tearDown(self):
        shutil.rmtree(self.TEST_DIR)

In [13]:
# load model and inspect output
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("The girl broke the vase with a basketball.")
output

2022-03-15 15:47:33,066 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-03-15 15:47:34,733 - WARNING - allennlp.common.model_card - lerc is not a registered model.
2022-03-15 15:47:35,603 - INFO - allennlp.common.plugins - Plugin allennlp_models available
2022-03-15 15:47:35,840 - INFO - cached_path - cache of https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz is up-to-date
2022-03-15 15:47:35,841 - INFO - allennlp.models.archival - loading archive file https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz from cache at /Users/felixdenheijer/.allennlp/cache/60314a853eb0aaa774d176d878c62469d49872feb4f2bfd071a75c77f6d76707.1b91cc27e347f2df04ce771a304bee2b70a2c487626b67e277d44c593b868c25
2022-03-15 15:47:35,842 - INFO - allennlp.models.archival - extracting archive file /Users/felixdenheijer/.allennlp/cache/60314a853eb0aaa774d176d878c62469d49872feb4f2bfd071a75c77f6d76707.1b91cc27e347f2df04ce771a304be

{'verbs': [{'verb': 'broke',
   'description': '[ARG0: The girl] [V: broke] [ARG1: the vase] [ARG2: with a basketball] .',
   'tags': ['B-ARG0',
    'I-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'B-ARG2',
    'I-ARG2',
    'I-ARG2',
    'O']}],
 'words': ['The',
  'girl',
  'broke',
  'the',
  'vase',
  'with',
  'a',
  'basketball',
  '.']}

In [13]:
### added by pia ###

def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [14]:
d = ["Ben Reynolds was killed by someone last night.","The killer killed the victim with a knife." ]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'killed',
     'description': '[ARG1: Ben Reynolds] was [V: killed] [ARG0: by someone] [ARGM-TMP: last night] .',
     'tags': ['B-ARG1',
      'I-ARG1',
      'O',
      'B-V',
      'B-ARG0',
      'I-ARG0',
      'B-ARGM-TMP',
      'I-ARGM-TMP',
      'O']}],
   'words': ['Ben',
    'Reynolds',
    'was',
    'killed',
    'by',
    'someone',
    'last',
    'night',
    '.']},
  {'verbs': [{'verb': 'killed',
     'description': '[ARG0: The killer] [V: killed] [ARG1: the victim] [ARG2: with a knife] .',
     'tags': ['B-ARG0',
      'I-ARG0',
      'B-V',
      'B-ARG1',
      'I-ARG1',
      'B-ARG2',
      'I-ARG2',
      'I-ARG2',
      'O']}],
   'words': ['The',
    'killer',
    'killed',
    'the',
    'victim',
    'with',
    'a',
    'knife',
    '.']}],
 array([1., 1.]))

In [15]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

# Checklist examples

In [16]:
# Helper function to extract target argument

def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set


# Helper function to display failures

def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure



# Test arg1 with names

In [17]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name'], meta['last_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

## Western names

In [18]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("Someone killed {first_name} {last_name} last night.", meta=True, nsamples=10)

print(type(t))

for k, v in t.items():
    print(k, v)

<class 'checklist.editor.MunchWithAdd'>
meta [{'first_name': 'Benjamin', 'last_name': 'Rose'}, {'first_name': 'Kathryn', 'last_name': 'Roberts'}, {'first_name': 'Suzanne', 'last_name': 'Collins'}, {'first_name': 'Jennifer', 'last_name': 'King'}, {'first_name': 'Henry', 'last_name': 'King'}, {'first_name': 'Margaret', 'last_name': 'Robinson'}, {'first_name': 'Jill', 'last_name': 'Crawford'}, {'first_name': 'Walter', 'last_name': 'Jones'}, {'first_name': 'Alan', 'last_name': 'Davies'}, {'first_name': 'Diana', 'last_name': 'Ryan'}]
data ['Someone killed Benjamin Rose last night.', 'Someone killed Kathryn Roberts last night.', 'Someone killed Suzanne Collins last night.', 'Someone killed Jennifer King last night.', 'Someone killed Henry King last night.', 'Someone killed Margaret Robinson last night.', 'Someone killed Jill Crawford last night.', 'Someone killed Walter Jones last night.', 'Someone killed Alan Davies last night.', 'Someone killed Diana Ryan last night.']


In [16]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg1_name_default_position', expect=expect_arg1)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 10 examples
Test cases:      10
Fails (rate):    0 (0.0%)


In [17]:
# the test object:
#dir(test)
for k, v in test.results.items():
    print(k, v)

preds [{'verbs': [{'verb': 'killed', 'description': '[ARG0: Someone] [V: killed] [ARG1: Richard Ford] [ARGM-TMP: last night] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'I-ARGM-TMP', 'O']}], 'words': ['Someone', 'killed', 'Richard', 'Ford', 'last', 'night', '.']}, {'verbs': [{'verb': 'killed', 'description': '[ARG0: Someone] [V: killed] [ARG1: Jay Stevens] [ARGM-TMP: last night] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'I-ARGM-TMP', 'O']}], 'words': ['Someone', 'killed', 'Jay', 'Stevens', 'last', 'night', '.']}, {'verbs': [{'verb': 'killed', 'description': '[ARG0: Someone] [V: killed] [ARG1: Kathryn Morris] [ARGM-TMP: last night] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'I-ARGM-TMP', 'O']}], 'words': ['Someone', 'killed', 'Kathryn', 'Morris', 'last', 'night', '.']}, {'verbs': [{'verb': 'killed', 'description': '[ARG0: Someone] [V: killed] [ARG1: Lucy Bennett] [ARGM-TMP: last night] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1',

In [18]:
test.summary(format_example_fn=format_srl)

Test cases:      10
Fails (rate):    0 (0.0%)


In [20]:
editor = Editor()
t = editor.template("{first_name} {last_name} was killed by someone last night.", meta=True, nsamples=10)
test = MFT(**t, expect=expect_arg1)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 10 examples
Test cases:      10
Fails (rate):    1 (10.0%)

Example fails:
[ARGM-DIS: Andrea] [ARG1: Richardson] was [V: killed] [ARG0: by someone] [ARGM-TMP: last night] .
----


In [21]:
editor = Editor()
t = editor.template("It was {first_name} {last_name} who got killed last night.", meta=True, nsamples=10)
test = MFT(**t, expect=expect_arg1)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 10 examples
Test cases:      10
Fails (rate):    0 (0.0%)


## Non-western names

In [22]:
first = [x.split()[0] for x in editor.lexicons.male_from.Vietnam +  editor.lexicons.female_from.Vietnam]
last = [x.split()[0] for x in editor.lexicons.last_from.Vietnam]


In [25]:
t = editor.template("Someone killed {first_name} {last_name}  last night.", first_name=first, last_name=last, meta=True, nsamples=10)
test = MFT(**t, expect=expect_arg1)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 10 examples
Test cases:      10
Fails (rate):    1 (10.0%)

Example fails:
[ARG0: Someone] [V: killed] [ARG1: Walter] Dương [ARGM-TMP: last night] .
----


In [24]:
t = editor.template("{first_name} {last_name} was killed by someone last night.", first_name=first, last_name=last, meta=True, nsamples=10)
test = MFT(**t, expect=expect_arg1)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 10 examples
Test cases:      10
Fails (rate):    1 (10.0%)

Example fails:
[V: Can] [ARG1: Ngo] was killed by someone last night .
----


In [27]:
t = editor.template("It was {first_name} {last_name} who was killed last night.", first_name=first, last_name=last, meta=True, nsamples=10)
test = MFT(**t, expect=expect_arg1)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 10 examples
Test cases:      10
Fails (rate):    3 (30.0%)

Example fails:
It was Charlotte [ARG1: Mã] [R-ARG1: who] was [V: killed] [ARGM-TMP: last night] .
----
It was Gioan Diem [R-ARG1: who] was [V: killed] [ARGM-TMP: last night] .
----
It was Mai [ARG1: Trương] [R-ARG1: who] was [V: killed] [ARGM-TMP: last night] .
----


# Test instrument

In [28]:
def found_arg2_instrument(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1
    
    instrument = set(meta['instrument'].split(' '))
    arg_3 = get_arg(pred, arg_target='ARG2')

    if arg_3 == instrument:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg2 = Expect.single(found_arg2_instrument)

In [29]:
t = editor.template("The girl broke the vase {instrument}.", instrument=['with a football'],  meta=True, nsamples=1)
test = MFT(**t, expect=expect_arg2)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 1 examples
Test cases:      1
Fails (rate):    0 (0.0%)


In [30]:
t = editor.template("The girl opened the envenlope {instrument}.", instrument=['with a knife'],  meta=True, nsamples=1)
test = MFT(**t, expect=expect_arg2)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 1 examples
Test cases:      1
Fails (rate):    1 (100.0%)

Example fails:
[ARG0: The girl] [V: opened] [ARG1: the envenlope] [ARG3: with a knife] .
----


In [31]:
t = editor.template("{instrument} opened the envelope", instrument=['The knife'],  meta=True, nsamples=1)
test = MFT(**t, expect=expect_arg2)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 1 examples
Test cases:      1
Fails (rate):    1 (100.0%)

Example fails:
[ARG0: The knife] [V: opened] [ARG1: the envelope]
----


In [32]:
t = editor.template("{instrument} shut the door.", instrument=['The wind'],  meta=True, nsamples=1)
test = MFT(**t, expect=expect_arg2)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 1 examples
Test cases:      1
Fails (rate):    1 (100.0%)

Example fails:
[ARG0: The wind] [V: shut] [ARG1: the door] .
----


In [33]:
t = editor.template("{instrument} broke the window.", instrument=['The ball'],  meta=True, nsamples=1)
test = MFT(**t, expect=expect_arg2)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

Predicting 1 examples
Test cases:      1
Fails (rate):    1 (100.0%)

Example fails:
[ARG0: The ball] [V: broke] [ARG1: the window] .
----


# Experimenting with other test types: DIR

In [55]:
# dir(editor)
# help(editor.template)

In [188]:
import re
def change_agent_instrument(x, meta=True, *args, **kwargs):
    agents = ['John', 'Mary']
    instruments = ['The hammer'] #, 'The hammer']
    
    ret = []
    ret_meta = []
    
    for a in agents:
        if re.search(r'\b%s\b' % a, x):
            for i in instruments:
                ret.append(re.sub(r'\b%s\b' % a, i, x))
                ret_meta.append((a, i))
                
#             ret.extend([re.sub(r'\b%s\b' % a, i, x) for i in instruments])
#             ret_meta.extend([(a, i) for i in instruments])
    if meta:
        return ret, ret_meta
    else:
        return ret
    
    
    
def change_instrument_mod(x, meta=True, *args, **kwargs):
    instruments = ['the hammer', 'the key', 'the knife'] #, 'The hammer']
    mods = ['the flower'] #, 'the stripes', 'the stain']
    
    ret = []
    ret_meta = []
    
    for i in instruments:
        if re.search(r'\b%s\b' % i, x):
            for m in mods:
                ret.append(re.sub(r'\b%s\b' % i, m, x))
                ret_meta.append((i, m))
                
#             ret.extend([re.sub(r'\b%s\b' % a, i, x) for i in instruments])
#             ret_meta.extend([(a, i) for i in instruments])
    if meta:
        return ret, ret_meta
    else:
        return ret
    
    
# create test

def get_arg_span(pred, target_span=[]):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if w in target_span:
            arg_list.append(arg)
    return arg_list


In [185]:


def compare_spans(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    
    sp_orig = meta[0].split(' ')
    sp_pred = meta[1].split(' ')
    
    
    l_orig = set(get_arg_span(orig_pred, sp_orig))
    l_pred = set(get_arg_span(pred, sp_pred))
    
    if l_orig == l_pred:
        pass_ = False
    else:
        pass_ = True
    
    
    return pass_

expect_fn = Expect.pairwise(compare_spans)

## Agent v.s. instrument

In [197]:
# Prepare data

data = ['John broke the vase.',
       'Mary broke the window.',
       'John openend the door.',
       'Mary broke the log.',
       'John broke the glass.',
       'Mary broke the plate.']
        

t_p = Perturb.perturb(data, change_agent_instrument, 
                      meta=True, keep_original=True, n_samples=1)

# Modify the structure of the meta-data so they can be used for this test
new_meta=[]
for m in t_p.meta:
    new_meta.append(m[1])
t_p.meta = new_meta


In [ ]:
# run test
test = DIR(**t_p, expect = expect_fn)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

In [191]:
# inspect results
# ask yourself: is this test useful? what can it do? what can't it do?
for p in test.results.preds[:2]:
    print(p)
    print()

[{'verbs': [{'verb': 'broke', 'description': '[ARG0: John] [V: broke] [ARG1: the vase] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O']}], 'words': ['John', 'broke', 'the', 'vase', '.']}
 {'verbs': [{'verb': 'broke', 'description': '[ARG0: The hammer] [V: broke] [ARG1: the vase] .', 'tags': ['B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O']}], 'words': ['The', 'hammer', 'broke', 'the', 'vase', '.']}]

[{'verbs': [{'verb': 'broke', 'description': '[ARG0: Mary] [V: broke] [ARG1: the window] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O']}], 'words': ['Mary', 'broke', 'the', 'window', '.']}
 {'verbs': [{'verb': 'broke', 'description': '[ARG0: The hammer] [V: broke] [ARG1: the window] .', 'tags': ['B-ARG0', 'I-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O']}], 'words': ['The', 'hammer', 'broke', 'the', 'window', '.']}]



## Instrument v.s. NP modifier

In [198]:

data = ['John broke the vase with the hammer.',
       'Mary broke the window with the hammer.',
       'John openend the door with the key.',
       'John broke the glass with the hammer.',
       'Mary openend the envelope with the knife.']
        

In [199]:
t_p = Perturb.perturb(data, change_instrument_mod, 
                      meta=True, keep_original=True, n_samples=1)
new_meta=[]
for m in t_p.meta:
    new_meta.append(m[1])
t_p.meta = new_meta


t_p

MunchWithAdd({'data': [['John broke the vase with the hammer.', 'John broke the vase with the flower.'], ['Mary broke the window with the hammer.', 'Mary broke the window with the flower.'], ['John openend the door with the key.', 'John openend the door with the flower.'], ['John broke the glass with the hammer.', 'John broke the glass with the flower.'], ['Mary openend the envelope with the knife.', 'Mary openend the envelope with the flower.']], 'meta': [('the hammer', 'the flower'), ('the hammer', 'the flower'), ('the key', 'the flower'), ('the hammer', 'the flower'), ('the knife', 'the flower')]})

In [200]:
test = DIR(**t_p, expect = expect_fn)
print('running')
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)

running
Predicting 10 examples
Test cases:      5
Fails (rate):    3 (60.0%)

Example fails:
[ARGM-MNR: John] [V: openend] [ARG1: the door] [ARGM-MNR: with the key] .
[ARGM-MNR: John] [V: openend] [ARG1: the door] [ARGM-MNR: with the flower] .

----
[ARG0: John] [V: broke] [ARG1: the vase] [ARG2: with the hammer] .
[ARG0: John] [V: broke] [ARG1: the vase] [ARG2: with the flower] .

----
[ARG0: Mary] [V: openend] [ARG1: the envelope] [ARGM-MNR: with the knife] .
[ARG0: Mary] [V: openend] [ARG1: the envelope] [ARGM-MNR: with the flower] .

----


In [202]:
for pred in test.results.preds:
    print(pred)
    print()

[{'verbs': [{'verb': 'broke', 'description': '[ARG0: John] [V: broke] [ARG1: the vase] [ARG2: with the hammer] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']}], 'words': ['John', 'broke', 'the', 'vase', 'with', 'the', 'hammer', '.']}
 {'verbs': [{'verb': 'broke', 'description': '[ARG0: John] [V: broke] [ARG1: the vase] [ARG2: with the flower] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']}], 'words': ['John', 'broke', 'the', 'vase', 'with', 'the', 'flower', '.']}]

[{'verbs': [{'verb': 'broke', 'description': '[ARG0: Mary] [V: broke] [ARG1: the window] [ARG2: with the hammer] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']}], 'words': ['Mary', 'broke', 'the', 'window', 'with', 'the', 'hammer', '.']}
 {'verbs': [{'verb': 'broke', 'description': '[ARG0: Mary] [V: broke] [ARG1: the window] [ARGM-MNR: with the flower] .', 'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARGM-MNR